# Projet de session - IFT712

Configuration

In [34]:
import data_manager.data_manager
import data_manager.data_organizer

In [35]:
# Load the database.
db = data_manager.data_manager.DatabaseManager('')
db.load()
# Get the datasets.
dataset = db.get_train_dataset().drop('id', axis=1)

In [36]:
do = data_manager.data_organizer.DataOrganizer()
do.shuffle(dataset, dataset.columns)
do.standardize(dataset, dataset.columns[1:])
do.numerize(dataset, dataset.columns[0])

In [37]:
# Split the dataset in 70%/30%.
training_dataset = dataset[:int(dataset.shape[0] * 0.7)]
test_dataset = dataset[int(dataset.shape[0] * 0.7):]

# Separate the inputs from the targets.
training_inputs = training_dataset.drop('species', axis=1)
training_targets = training_dataset['species']

test_inputs = test_dataset.drop('species', axis=1)
test_targets = test_dataset['species']

In [38]:
from sklearn.svm import SVC

cl = SVC()
cl.fit(training_inputs, training_targets)
cl.score(test_inputs, test_targets)

0.9562289562289562